# SVD

[Relationship between PCA and SVD](https://stats.stackexchange.com/a/134283)

$\large \mathbf{X} = {\begin{pmatrix} x_{1,1}&\cdots&x_{1,p} \\ \vdots & \ddots &\vdots \\ x_{n,1}&\cdots&x_{n,p} \end{pmatrix} }_{n\times p} = {\begin{pmatrix} X_1 & \cdots&X_p \end{pmatrix} }_{n\times p}$ where $n$ is the number of samples and $p$ is the number of features.
<br/><br/>
$\large \mathrm{Cov}(X) = {\begin{pmatrix} \mathrm{cov}[X_1, X_1] & \cdots & \mathrm{cov}[X_1, X_p] \\ \vdots & \ddots &\vdots \\ \mathrm{cov}[X_p, X_1] & \cdots & \mathrm{cov}[X_p, X_p]\end{pmatrix}}_{p\times p} = {\begin{pmatrix} \mathrm{var}[X_1] & \cdots & \mathrm{cov}[X_1, X_p] \\ \vdots & \ddots &\vdots \\ \mathrm{cov}[X_p, X_1] & \cdots & \mathrm{var}[X_p]\end{pmatrix}}_{p\times p}$

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity

$\Large \bullet \textbf{ Gram-Schmidt process}$

$\large \displaystyle \mathrm{V} = \begin{pmatrix} \mathbf{v}_1 &\mathbf{v}_2 & \cdots & \mathbf{v}_p \end{pmatrix}$
, where  $\mathbf{v}_i$ for $i \in \{1,2,\dots,p\}$ are column vectors.

$\large \begin{align}  \mathbf{u}_1 &= \mathbf{v}_1 \\ \mathbf{u}_2 &= \mathbf{v}_2 - \frac{\left\langle \mathbf{u}_1, \mathbf{v}_2 \right\rangle}{\left\langle \mathbf{u}_1, \mathbf{u}_1 \right\rangle} \mathbf{u}_1\\ \mathbf{u}_3 &= \mathbf{v}_3- \frac{\left\langle \mathbf{u}_2, \mathbf{v}_3 \right\rangle}{\left\langle \mathbf{u}_2, \mathbf{u}_2\right\rangle} \mathbf{u}_2 - \frac{\left\langle \mathbf{u}_1, \mathbf{v}_3 \right\rangle}{\left\langle \mathbf{u}_1, \mathbf{u}_1\right\rangle} \mathbf{u}_1\\ &\ \vdots \nonumber \\\mathbf{u}_p &= \mathbf{v}_p - \sum^{p-1}_{i=1} \frac{\langle \mathbf{u}_i, \mathbf{v}_p \rangle}{\left\langle \mathbf{u}_i, \mathbf{u}_i\right\rangle} \mathbf{u}_i\\
\end{align} $

The orthogonal column vectors matrix is

$\large \therefore \mathrm{U} = \begin{pmatrix} \mathbf{u}_1 & \mathbf{u}_2 &\cdots & \mathbf{u}_p \end{pmatrix}$

In [56]:
class gram_schmidt:
    '''def __init__(self, X):
        self.X = X.astype(float)'''
        
    def _proj(self, u, v):
        return ((u.T).dot(v) / (u.T).dot(u))*u
    
    def fit_transform(self, X, col_vec = True, normal = True):
        #X = X.astype(float)
        if col_vec:
            mat = X.copy()
        else:
            mat = (X.T).copy()
        
        N = mat.shape[1]
        mat_orth = np.array([]).reshape(mat.shape[0], -1)
        for n in range(N):
            u = mat[:, n:n+1].copy()
            if n ==0:
                mat_orth = np.hstack((mat_orth,u))
            else:
                for i in range(n):
                    u -= self._proj(mat_orth[:, i:i+1], mat[:, n:n+1])
                mat_orth = np.hstack((mat_orth,u))
        
        if normal:
            result = mat_orth / np.linalg.norm(mat_orth, axis=0)
            if col_vec:
                return result
            else:
                return result.T
        else:
            if col_vec:
                return mat_orth
            else:
                return mat_orth.T

In [308]:
def svd(data, reduced='thin'):
    dim = data.shape[1]
    rank = np.linalg.matrix_rank(data)
    relu = np.vectorize(lambda x: x if x>=0 else .0)
    
    eval_u, evec_u = np.linalg.eig(data.dot(data.T))
    eval_v, evec_v = np.linalg.eig(data.T.dot(data))
    
    gs = gram_schmidt()
    #evec_u_gs = gs.fit_transform(np.real(evec_u))
    #evec_v_gs = gs.fit_transform(np.real(evec_v))
    
    s = eval_v.copy()
    s = np.sqrt(relu(s))
    s1 = np.sort(s)[::-1]
    if dim > rank:
        s1[-(dim-rank):] = 0
     
    u_idx = np.sqrt(relu(np.real(eval_u))).argsort()[-dim:][::-1]
    if reduced=='thin':
        evec_u = evec_u[:, u_idx ]
        S = np.eye(dim)
        S *= s1     
    else:        
        evec_u = np.hstack((evec_u[:, u_idx ], np.delete(evec_u, u_idx, axis=1)))
        S = np.zeros(data.shape)
        np.fill_diagonal(S, s1)
    
    u = gs.fit_transform(np.real(evec_u))
    
    v_idx = np.sqrt(relu(np.real(eval_v))).argsort()[-dim:][::-1]
    evec_v = evec_v[:, v_idx ]
    v = gs.fit_transform(np.real(evec_v))
    return u, S, v

In [309]:
uu, ss, vv = svd(data_x_std, reduced='full')

In [310]:
uu[:,:4]

array([[-0.0242664 , -0.00028648, -0.02286707, -0.04226507],
       [ 0.02300186, -0.03048532,  0.03569275,  0.01812984],
       [ 0.04269872,  0.03298193, -0.03038933, -0.00589413],
       ...,
       [-0.02825438,  0.00468142, -0.00570868,  0.04847292],
       [-0.03299276, -0.05398932, -0.01271873, -0.01253839],
       [-0.01164199,  0.01096364, -0.00040428, -0.04319016]])

In [311]:
u[:,:4]

array([[-0.0242664 ,  0.00028648,  0.02286707,  0.04226507],
       [ 0.02300186,  0.03048532, -0.03569275, -0.01812984],
       [ 0.04269872, -0.03298193,  0.03038933,  0.00589413],
       ...,
       [-0.02825438, -0.00468142,  0.00570868, -0.04847292],
       [-0.03299276,  0.05398932,  0.01271873,  0.01253839],
       [-0.01164199, -0.01096364,  0.00040428,  0.04319016]])

In [312]:
vv[0,:]

array([ 0.67673188,  0.181047  , -0.10787089, -0.70542173])

In [313]:
v.T[0,:]

array([-0.67673188,  0.181047  , -0.10787089,  0.70542173])

In [256]:
np.allclose(abs(uu[:,:20]), abs(u[:,:20]))

True

In [257]:
np.allclose(abs(vv), abs(v.T))

True

In [258]:
np.allclose(ss.diagonal(), s)

True

In [265]:
ss.shape

(20, 20)

In [263]:
uu.dot(ss.dot(vv))

array([[ 0.5647267 , -0.47680363, -0.78627148, ...,  1.08572554,
        -2.6671056 ,  1.15872028],
       [ 1.44679163, -0.45103722,  1.08659197, ..., -1.28457032,
        -1.04955323,  0.6106934 ],
       [-3.19007586,  0.88998617,  2.55359432, ..., -0.84053839,
         0.99574463, -1.13585373],
       ...,
       [-1.33127016, -0.52435115,  0.86350697, ...,  0.5527819 ,
        -0.65783193, -1.74325353],
       [ 2.15374311,  0.04029905, -0.20825025, ..., -1.44913037,
         1.85544038, -0.89856518],
       [ 0.2187607 ,  1.13683451, -0.81481309, ..., -0.04568927,
        -0.16913704,  0.96884685]])

In [294]:
np.allclose(data_x_std, uu.dot(ss.dot(vv)))

False

In [295]:
data_x_std[0]

array([ 1.35587159, -0.22021195, -0.51007816,  0.74336135])

In [296]:
uu.dot(ss.dot(vv.T))[0]

array([-0.54566582,  1.39833053, -0.00319979, -0.66824935])

In [297]:
u.dot(zzzz)

array([[ 1.35587159, -0.22021195, -0.51007816,  0.74336135],
       [-0.60635073, -0.08883097,  0.07661929, -1.63845147],
       [-1.15491588,  0.57826209, -1.2426252 ,  0.96683847],
       ...,
       [-0.39561355, -0.58737837,  1.50776079,  0.45752966],
       [ 1.29201867, -1.62658271,  0.10409944, -0.52231705],
       [ 1.09287156,  0.54004342, -0.50957456,  0.36172784]])

In [288]:
sss = np.zeros(data_x_std.shape)
np.fill_diagonal(sss, s)
zzzz = sss.dot(v)
np.allclose(data_x_std, u.dot(zzzz))

True

In [248]:
np.allclose(ss.dot(vv.T), zzzz)

False

In [250]:
zzzz

array([[  2.04093653,  11.85705953,   8.88837402, ...,  -8.96838802,
          6.35307545,  -7.33362411],
       [-14.63528048,   5.92302425,   9.16221672, ..., -10.35639044,
        -15.71096592,  -0.37098284],
       [-12.31918916,   0.30021872,   2.81180782, ...,  -0.25297427,
          5.67919043,   3.33489377],
       ...,
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

In [249]:
ss.dot(vv.T)

array([[  2.04093653,  11.85705953,   8.88837402, ...,  -8.96838802,
          6.35307545,  -7.33362411],
       [-14.63528048,   5.92302425,   9.16221672, ..., -10.35639044,
        -15.71096592,  -0.37098284],
       [ 12.31918916,  -0.30021872,  -2.81180782, ...,   0.25297427,
         -5.67919043,  -3.33489377],
       ...,
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

# Codes

In [274]:
data = make_classification(n_features = 4, n_redundant=0,
                           n_samples=10**3, weights=[0.9], random_state= 42, )

In [275]:
data_x = data[0]
data_y = data[1]

$\large \displaystyle \mathbf{X}_{\normalsize standardized} = \begin{pmatrix} \frac{X_1 -  \mu_1}{\sigma_1} & \frac{X_2 -  \mu_2}{\sigma_2} & \cdots &\frac{X_p -  \mu_p}{\sigma_p}  \end{pmatrix} $
, where $\mu_i$ and $\sigma_i$ are mean and standard deviation of the vector $X_i$ for $i \in \{1,2,\cdots,p\}$.

Therefore, the means of each column of matrix $\mathbf{X}_{\normalsize standardized}$ are all 0 and standard deviations are all 1.

In [276]:
scaler = StandardScaler()
data_x_std = scaler.fit_transform(data_x)

$\large X_{n\times p} = U\; \Sigma\; V $

$U_{n\times n} =$ orthonormal eigenvectors of $XX^* $

$\Sigma_{n\times p} =$  the square roots of the non-negative eigenvalues of both $X X^*$ and $X^* X $

${V^*}_{p\times p} =$ orthonormal eigenvectors of $X^*X$

In [277]:
u, s, v = np.linalg.svd(data_x_std, full_matrices=True, )

In [278]:
np.allclose(v.dot(v.T), np.eye(v.shape[0]))

True

In [279]:
np.linalg.det(v)

0.9999999999999989

In [146]:
S = np.zeros(data_x_std.shape, float)
np.fill_diagonal(S, s)

In [147]:
np.allclose(np.dot(u, np.dot(S, v)), data_x_std )

True

$\large \therefore X \simeq U \Sigma V$

In [148]:
eval_u, evec_u = np.linalg.eig(data_x_std.dot(data_x_std.T))
eval_v, evec_v = np.linalg.eig(data_x_std.T.dot(data_x_std))

In [171]:
gs = gram_schmidt()
evec_u_gs = gs.fit_transform(np.real(evec_u))
evec_v_gs = gs.fit_transform(evec_v)

In [130]:
np.linalg.matrix_rank(data_x_std)

18

In [149]:
s

array([36.62298228, 35.4665271 , 34.65973861, 33.65057709, 33.25522622,
       33.02768341, 32.93607305, 32.56124304, 32.18629137, 31.71929746,
       31.24794918, 31.07841663, 30.70064104, 30.31144614, 29.76964458,
       29.24344091, 29.02919187, 27.96984113, 27.90012513, 27.10989188])

In [151]:
np.sqrt(eval_v)

array([36.62298228, 35.4665271 , 34.65973861, 27.10989188, 27.90012513,
       27.96984113, 29.02919187, 29.24344091, 29.76964458, 33.65057709,
       30.31144614, 30.70064104, 31.07841663, 31.24794918, 31.71929746,
       32.18629137, 33.25522622, 33.02768341, 32.93607305, 32.56124304])

In [177]:
u[0,:20]

array([-0.02627431, -0.06248842,  0.01506608,  0.00412621,  0.03052269,
       -0.01413369, -0.01447024, -0.0337205 , -0.05524327,  0.0685282 ,
       -0.03114748, -0.09625528, -0.02225258,  0.02172116, -0.01944328,
       -0.00494595,  0.02787485, -0.00633242, -0.00782038,  0.02615544])

In [217]:
zzz = np.arange(1,26).reshape(-1,5)
np.delete(zzz, [0,1], axis=1)

array([[ 3,  4,  5],
       [ 8,  9, 10],
       [13, 14, 15],
       [18, 19, 20],
       [23, 24, 25]])

In [211]:
evec_u_gs[:, ~(np.sqrt(np.vectorize(lambda x: x if x>=0 else .0)(np.real(eval_u))).argsort()[-20:][::-1]) ].shape

(1000, 20)

In [181]:
evec_u_gs[:, np.sqrt(np.vectorize(lambda x: x if x>=0 else .0)(np.real(eval_u))).argsort()[-20:][::-1] ][0]

array([ 0.02627431, -0.06248842,  0.01506608,  0.00412621,  0.03052269,
        0.01413369,  0.01447024,  0.0337205 , -0.05524327,  0.0685282 ,
       -0.03114748,  0.09625528,  0.02225258, -0.02172116,  0.01944328,
       -0.00494595,  0.02787485, -0.00633242, -0.00782038, -0.02615544])

In [183]:
np.allclose(abs(evec_u_gs[:, np.sqrt(np.vectorize(lambda x: x if x>=0 else .0)(np.real(eval_u))).argsort()[-20:][::-1] ][0]) , abs(u[0,:20]))

True

In [167]:
np.sqrt(np.vectorize(lambda x: x if x>=0 else .0)(np.real(eval_u))).argsort()[-20:][::-1]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 15,
       19, 18, 17], dtype=int64)

In [166]:
np.sqrt(np.vectorize(lambda x: x if x>=0 else .0)(np.real(eval_u)))

array([3.66229823e+01, 3.54665271e+01, 3.46597386e+01, 3.36505771e+01,
       3.32552262e+01, 3.30276834e+01, 3.29360731e+01, 3.25612430e+01,
       3.21862914e+01, 3.17192975e+01, 3.12479492e+01, 3.10784166e+01,
       3.07006410e+01, 3.03114461e+01, 2.97696446e+01, 2.90291919e+01,
       2.92434409e+01, 2.71098919e+01, 2.79001251e+01, 2.79698411e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.80870126e-07, 0.00000000e+00, 0.00000000e+00,
       3.44837393e-07, 3.03838628e-07, 3.03838628e-07, 2.01318457e-07,
       2.01318457e-07, 3.12388595e-07, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.53996837e-07, 2.49052843e-07,
       2.49052843e-07, 8.20162314e-08, 8.20162314e-08, 2.21401845e-07,
       2.21401845e-07, 0.00000000e+00, 0.00000000e+00, 1.96324965e-07,
       1.96324965e-07, 0.00000000e+00, 0.00000000e+00, 2.37474021e-07,
       2.37474021e-07, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [157]:
np.isin(np.sqrt(np.real(eval_u)), np.sqrt(eval_v))

<ipython-input-157-e117c599922b>:1: RuntimeWarning: invalid value encountered in sqrt
  np.isin(np.sqrt(np.real(eval_u)), np.sqrt(eval_v))


array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [139]:
eval_v[[eval_v.argsort()]]

array([19, 18,  8,  9, 13, 15, 16, 17, 14, 12, 11, 10,  7,  6,  5,  4,  3,
        2,  1,  0], dtype=int64)

In [120]:
np.sort(np.nan_to_num(np.sqrt(np.real(eval_v)), 0))

array([27.10989188, 27.90012513, 27.96984113, 29.02919187, 29.24344091,
       29.76964458, 30.31144614, 30.70064104, 31.07841663, 31.24794918,
       31.71929746, 32.18629137, 32.56124304, 32.93607305, 33.02768341,
       33.25522622, 33.65057709, 34.65973861, 35.4665271 , 36.62298228])

In [119]:
np.sort(np.nan_to_num(np.sqrt(np.real(eval_u)), 0))[::-1][:20]

<ipython-input-119-a57b782d93bb>:1: RuntimeWarning: invalid value encountered in sqrt
  np.sort(np.nan_to_num(np.sqrt(np.real(eval_u)), 0))[::-1][:20]


array([36.62298228, 35.4665271 , 34.65973861, 33.65057709, 33.25522622,
       33.02768341, 32.93607305, 32.56124304, 32.18629137, 31.71929746,
       31.24794918, 31.07841663, 30.70064104, 30.31144614, 29.76964458,
       29.24344091, 29.02919187, 27.96984113, 27.90012513, 27.10989188])

In [89]:
np.fill_diagonal(np.zeros((6,4)), np.arange(1,5))

In [92]:
xx = np.zeros((6,4))
np.fill_diagonal(xx, np.arange(1,5))
xx.dot(np.arange(1,17).reshape(-1,4))

array([[ 1.,  2.,  3.,  4.],
       [10., 12., 14., 16.],
       [27., 30., 33., 36.],
       [52., 56., 60., 64.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [93]:
np.arange(1,17).reshape(-1,4)


array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]])

In [75]:
u[0,:15]

array([ 0.01828462,  0.04724048, -0.02791569, -0.00476045, -0.01081699,
       -0.01415586,  0.04619612, -0.03305375,  0.01476144, -0.05566156,
       -0.02595789, -0.00378986, -0.0323959 ,  0.00981624,  0.00089818])

In [86]:
np.real(evec_u[:,np.sqrt(abs(np.real(eval_v))).argsort()[::-1]][0,:15])

array([ 0.01828462,  0.04724048, -0.02791569, -0.00476045,  0.01081699,
        0.01415586, -0.04619612, -0.03305375,  0.02595789,  0.02238961,
       -0.01663035, -0.0323959 , -0.00089818, -0.00260253, -0.00981624])

In [59]:
np.allclose(evec_v_gs.dot(evec_v_gs.T), np.eye(evec_v_gs.shape[0]))

True

In [161]:
np.sqrt(abs(np.real(eval_v))).argsort()

array([19, 18,  4,  5,  6,  7,  8, 10, 11, 12, 13, 17, 16, 15, 14,  9,  3,
        2,  1,  0], dtype=int64)

In [35]:
s

array([5.54205355e+01, 3.45049635e+01, 3.43763398e+01, 3.39458464e+01,
       3.37255154e+01, 3.32130138e+01, 3.28376444e+01, 3.25646897e+01,
       3.17946166e+01, 3.17083444e+01, 3.08858574e+01, 3.05148789e+01,
       2.99609330e+01, 2.98545964e+01, 2.95726288e+01, 2.92951048e+01,
       2.84758091e+01, 2.81185613e+01, 1.39203880e-14, 1.16748491e-14])

In [46]:
np.sort(np.sqrt(abs(eval_v)))[::-1]

array([5.54205355e+01, 3.45049635e+01, 3.43763398e+01, 3.39458464e+01,
       3.37255154e+01, 3.32130138e+01, 3.28376444e+01, 3.25646897e+01,
       3.17946166e+01, 3.17083444e+01, 3.08858574e+01, 3.05148789e+01,
       2.99609330e+01, 2.98545964e+01, 2.95726288e+01, 2.92951048e+01,
       2.84758091e+01, 2.81185613e+01, 4.60001772e-07, 3.13500387e-07])

In [42]:
np.sort(np.nan_to_num(np.sqrt(eval_v),0))[::-1]

<ipython-input-42-eeac0a74da89>:1: RuntimeWarning: invalid value encountered in sqrt
  np.sort(np.nan_to_num(np.sqrt(eval_v),0))[::-1]


array([5.54205355e+01, 3.45049635e+01, 3.43763398e+01, 3.39458464e+01,
       3.37255154e+01, 3.32130138e+01, 3.28376444e+01, 3.25646897e+01,
       3.17946166e+01, 3.17083444e+01, 3.08858574e+01, 3.05148789e+01,
       2.99609330e+01, 2.98545964e+01, 2.95726288e+01, 2.92951048e+01,
       2.84758091e+01, 2.81185613e+01, 4.60001772e-07, 0.00000000e+00])

In [81]:
v.T[3,:]

array([-1.09124406e-02,  3.53755625e-02, -2.23088454e-01,  3.08145422e-02,
        1.40965890e-01,  4.36320529e-01,  2.93179868e-01,  1.15110843e-01,
       -8.89513509e-02, -5.88986414e-01,  1.53103165e-01, -1.56757561e-01,
       -1.65388383e-01, -6.38952559e-02,  1.96094406e-01, -2.00332607e-01,
       -2.52898496e-01, -2.48292121e-01,  1.97553277e-16, -1.00982859e-16])

In [82]:
np.real(evec_v_gs[:,np.sqrt(abs(np.real(eval_v))).argsort()[::-1]])[3,:]

array([-1.09124406e-02,  3.53755625e-02,  2.23088454e-01, -3.08145422e-02,
        1.40965890e-01, -4.36320529e-01, -2.93179868e-01,  1.15110843e-01,
       -8.89513509e-02, -5.88986414e-01,  1.53103165e-01,  1.56757561e-01,
       -1.65388383e-01, -6.38952559e-02, -1.96094406e-01, -2.00332607e-01,
       -2.52898496e-01,  2.48292121e-01,  2.19644772e-17, -4.36892929e-16])

In [129]:
gs = gram_schmidt()
ev_orth = np.real(gs.orthonormal(ev, ))

In [132]:
v.T[0,:]

array([-2.29222837e-02,  5.05722912e-01, -6.56554623e-02,  1.11001843e-01,
       -2.38002695e-01,  8.32706762e-02,  1.72357613e-01,  8.64398786e-02,
       -1.29605480e-02,  4.39800060e-01, -3.06539027e-01,  3.62347473e-02,
       -2.50551049e-03, -1.59530740e-02, -2.07104101e-01,  2.14519904e-01,
        4.72252971e-01, -1.58942830e-01,  5.97211875e-17, -3.02034327e-17])

In [133]:
ev_orth[0,:]

array([-2.29222837e-02, -5.05722912e-01,  6.56554623e-02,  1.11001843e-01,
        1.58942830e-01,  4.72252971e-01,  2.14519904e-01, -2.07104101e-01,
        1.59530740e-02, -2.38002695e-01, -2.50551049e-03,  3.62347473e-02,
        3.06539027e-01,  4.39800060e-01, -8.32706762e-02, -1.72357613e-01,
       -8.64398786e-02,  1.29605480e-02,  1.56755213e-17, -8.81978737e-18])

In [124]:
np.linalg.norm(ev_orth[:,1])

1.0000000000000002

In [123]:
np.linalg.norm(v.T[:,1])

1.0000000000000002

In [96]:
np.isclose(v[:,:3],np.real(gs.orthonormal(ev, )[:,:3]))

array([[ True, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False]])

In [97]:
v.T[:,:3] 

array([[-0.02292228,  0.50572291, -0.06565546],
       [ 0.55962697,  0.05425466, -0.04748907],
       [-0.0203438 , -0.01816242, -0.51729607],
       [ 0.00109865, -0.1577597 , -0.07062832],
       [-0.04118024, -0.25275819, -0.01802699],
       [-0.47676204,  0.15049368, -0.05232398],
       [ 0.0125351 ,  0.09318769,  0.45878404],
       [ 0.01736969, -0.20223319, -0.02511544],
       [ 0.03330077,  0.05702416, -0.17681541],
       [-0.00782674, -0.28581579,  0.09669366],
       [-0.01586192,  0.22138756, -0.18911348],
       [ 0.01047088, -0.24002708, -0.27383818],
       [ 0.02324671,  0.06861466, -0.03325629],
       [ 0.05600357,  0.1658956 ,  0.48019901],
       [-0.37526174, -0.21968459,  0.11830003],
       [ 0.00791182, -0.35257137, -0.07828296],
       [-0.02624688,  0.30492018,  0.00310865],
       [-0.01452093,  0.1160483 , -0.10764729],
       [ 0.55590289, -0.05176735,  0.00266235],
       [-0.02001138,  0.27290482, -0.30312492]])

In [100]:
ev_orth

array([[-0.02292228, -0.50572291,  0.06565546],
       [ 0.55962697, -0.05425466,  0.04748907],
       [-0.0203438 ,  0.01816242,  0.51729607],
       [ 0.00109865,  0.1577597 ,  0.07062832],
       [-0.04118024,  0.25275819,  0.01802699],
       [-0.47676204, -0.15049368,  0.05232398],
       [ 0.0125351 , -0.09318769, -0.45878404],
       [ 0.01736969,  0.20223319,  0.02511544],
       [ 0.03330077, -0.05702416,  0.17681541],
       [-0.00782674,  0.28581579, -0.09669366],
       [-0.01586192, -0.22138756,  0.18911348],
       [ 0.01047088,  0.24002708,  0.27383818],
       [ 0.02324671, -0.06861466,  0.03325629],
       [ 0.05600357, -0.1658956 , -0.48019901],
       [-0.37526174,  0.21968459, -0.11830003],
       [ 0.00791182,  0.35257137,  0.07828296],
       [-0.02624688, -0.30492018, -0.00310865],
       [-0.01452093, -0.1160483 ,  0.10764729],
       [ 0.55590289,  0.05176735, -0.00266235],
       [-0.02001138, -0.27290482,  0.30312492]])

In [99]:
gs = gram_schmidt()
ev_orth = np.real(gs.orthonormal(ev, )[:,:3])

In [60]:
u

array([[-3.58069699e-02,  3.01698041e-02, -3.92649977e-03, ...,
         2.50448298e-02, -3.82197662e-02, -3.66543863e-02],
       [-5.22878445e-03, -3.06603742e-02,  1.55813512e-02, ...,
        -2.54367666e-02, -4.40285614e-02, -1.38019509e-02],
       [ 4.67882864e-03, -7.81796912e-02, -1.31807645e-02, ...,
        -3.06775784e-02,  3.01705551e-02, -1.05746981e-02],
       ...,
       [-2.34054071e-02, -6.97085261e-02,  8.71660084e-03, ...,
         9.75102822e-01,  7.05642576e-04,  2.80920160e-03],
       [-1.42460634e-02, -7.98736725e-03, -2.16604159e-02, ...,
         1.85549046e-03,  9.82551168e-01,  6.20014067e-03],
       [-1.94011010e-02,  2.70296071e-02,  4.82344894e-03, ...,
         6.53096767e-04,  5.39052629e-03,  9.84550389e-01]])

In [18]:
cov= np.cov(X_train_sc - X_train_sc.mean(axis=0), rowvar=False)
eigenvalues, eigenvectors = np.linalg.eig(cov)

In [109]:
u.dot(np.diag(s))

(800, 20)

In [104]:
xpca

array([[ 1.77774354, -0.58563607],
       [ 0.2612659 ,  0.59186165],
       [-0.23103433,  2.09704588],
       ...,
       [ 1.16470754,  2.06088412],
       [ 0.70823629,  0.47223255],
       [ 0.96628558, -1.09295898]])

In [101]:
(X_train_sc - X_train_sc.mean(axis=0)).dot(pca.components_.T)

array([[ 1.78141977, -0.94647922],
       [ 0.26013539,  0.96186926],
       [-0.23277473,  2.45263287],
       ...,
       [ 1.16443405,  2.18687769],
       [ 0.70875081,  0.2505776 ],
       [ 0.96521724, -0.84796578]])

In [79]:
np.allclose(xpca, X_train_sc.dot(pca.components_.T))

False

In [75]:
X_train_sc.var()

0.9999999999999993

In [71]:
pca.explained_variance_

array([3.0668676 , 0.93813866])

In [74]:
(X_train_sc - X_train_sc.mean(axis=0)).dot(pca.components_.T).var(axis=0, ddof=1)

array([3.0668676 , 0.93813866])

In [19]:
np.allclose(v[0], eigenvectors.T[0])

False

In [20]:
eigenvectors[0,1]

-0.5057229118627715

In [21]:
np.isclose(abs(v), abs(eigenvectors.T))

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [False, False, False, False],
       [False, False, False, False]])

In [20]:
np.allclose(v.T.dot(v), np.eye(v.shape[0]))

True

In [22]:
np.isclose(eigenvectors.T.dot(eigenvectors), np.eye(eigenvectors.shape[0]))

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True, False],
       [ True,  True, False,  True]])

In [26]:
X_train_sc.shape

(800, 4)

In [69]:
comp_idx = 0
cosine_similarity(v[comp_idx:comp_idx+1,:], eigenvectors[:,comp_idx].reshape(1,-1))

array([[-1.]])

In [85]:
((u.T).dot(u) - np.eye(800)).min()

-1.7763568394002505e-15

In [27]:
np.allclose((u.T).dot(u) , np.eye(u.shape[0]))

True

In [28]:
np.allclose((v.T).dot(v) , np.eye(v.shape[0]))

True

In [82]:
np.round((u.T).dot(u) - np.eye(800), 2)

array([[ 0., -0., -0., ...,  0.,  0.,  0.],
       [-0., -0., -0., ...,  0.,  0.,  0.],
       [-0., -0., -0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0., -0.,  0.],
       [ 0.,  0.,  0., ..., -0.,  0., -0.],
       [ 0.,  0.,  0., ...,  0., -0., -0.]])

In [77]:
np.round((v.T).dot(v), 2)

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0., -0.],
       [ 0.,  0.,  1.,  0.],
       [ 0., -0.,  0.,  1.]])

In [75]:
eigenvectors.dot( np.matrix(eigenvectors).H)

matrix([[ 1.01230942,  0.06288618, -0.03379583, -0.08403019],
        [ 0.06288618,  1.14011871,  0.01014754, -0.09029393],
        [-0.03379583,  0.01014754,  0.90831919, -0.11137876],
        [-0.08403019, -0.09029393, -0.11137876,  0.93925268]])

In [78]:
np.round((eigenvectors.T).dot(eigenvectors),2)

array([[ 1.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  1.  , -0.  , -0.  ],
       [ 0.  , -0.  ,  1.  ,  0.22],
       [ 0.  , -0.  ,  0.22,  1.  ]])

In [36]:
np.square(s)/(X_train_sc.shape[0]-1)

array([3.06686760e+00, 9.38138656e-01, 1.54891299e-31, 4.74019816e-32])

In [34]:
np.square(s)/(X_train_sc.shape[0]-1)

array([3.06686760e+00, 9.38138656e-01, 1.54891299e-31, 4.74019816e-32])

In [173]:
cov= np.cov(X_train_sc - X_train_sc.mean(axis=0), rowvar=False)
eigenvalues, eigenvectors = np.linalg.eig(cov)

In [183]:
np.isclose(np.round(eigenvectors.T.dot(eigenvectors),2),np.eye(20))

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ T

In [178]:
np.linalg.norm(pca.components_[1])

0.9999999999999997

In [8]:
eigenvalues, eigenvectors = np.linalg.eig(cov)

In [9]:
eigenvalues

array([3.06686760e+00, 9.38138656e-01, 9.63907292e-17, 2.74949812e-16])

In [65]:
pca.explained_variance_

array([3.0668676 , 0.93813866])

In [68]:
pca.explained_variance_ratio_

array([0.7657585, 0.2342415])

In [11]:
pca.components_

array([[ 0.37537327, -0.56343987,  0.47853489, -0.55913753],
       [-0.77887225,  0.17161566,  0.5645045 , -0.21269872]])

In [12]:
-(eigenvectors[:, :2].T)

array([[ 0.37537327, -0.56343987,  0.47853489, -0.55913753],
       [-0.77887225,  0.17161566,  0.5645045 , -0.21269872]])

In [13]:
xpca

array([[ 0.23675929,  0.79205581],
       [ 0.96430644, -1.33873515],
       [-2.64001914,  0.06398657],
       ...,
       [ 1.64837173, -0.88211564],
       [-2.08038828,  0.35941666],
       [ 0.01904595, -0.24080798]])

In [14]:
(X_train_sc - X_train_sc.mean(axis=0)).dot(pca.components_.T)

array([[ 0.23675929,  0.79205581],
       [ 0.96430644, -1.33873515],
       [-2.64001914,  0.06398657],
       ...,
       [ 1.64837173, -0.88211564],
       [-2.08038828,  0.35941666],
       [ 0.01904595, -0.24080798]])

1st and 2nd component vectors are orthonormal

In [15]:
np.round((eigenvectors[:, :2].T).dot(eigenvectors[:, :2]), 2)

array([[1., 0.],
       [0., 1.]])